In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import xlsxwriter
pd.options.display.float_format = '{:,.2f}'.format

## READING FILES

In [ ]:
date_respuesta = '2022-02-15'
date_respuesta = pd.to_datetime(date_respuesta)

In [ ]:
names_resp = ['report_id','status','report_date_local','amount','disbursement_batch_id','merchant_id','bank_account_number',
             'bank_name','bank_account_owner_name','execution_date','aplication_date','disbursement_tracking_key',
             'error_description']

In [ ]:
disp = pd.read_csv(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2022\02. FEBRUARY\SWAP 2022-02-15.csv')
rs1 = pd.read_excel(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\02. FEBRERO\Respuesta-Swap 2022-02-15.xlsx',
                    sheet_name='Report',names=names_resp,
                    converters={'bank_account_number':str,'disbursement_tracking_key':str,'report_id':str})
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})


# BIN

In [ ]:
bin1.head()

In [ ]:
bin1.shape

# ARCHIVO RESPUESTA

In [ ]:
rs1['Date_respuesta'] = date_respuesta
rs1['BIN'] = rs1['bank_account_number'].str[:3]
rs1.rename(columns ={'bank_account_number':'CLABE'}, inplace=True)
rs1['Metodo de dispersion'] = 'SWAP'
rs1.head()

In [ ]:
rs1.shape

In [ ]:
rs1['report_id'] = rs1['report_id'].str.strip()

In [ ]:
rs2 = rs1.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == "'085", "Banco_emisor"] = 'BANAMEX'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == 'CLIP', "Banco_emisor"] = 'STP'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '2021', "Banco_emisor"] = 'SANTANDER'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '2022', "Banco_emisor"] = 'SANTANDER'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '#N/D', "Banco_emisor"] = 'SANTANDER'
rs2.loc[rs2['disbursement_tracking_key'].str[:3] == '178', "Banco_emisor"] = 'BANAMEX'
rs2.loc[rs2['disbursement_tracking_key'].str[:2] == '17', "Banco_emisor"] = 'BANAMEX'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '1.78', "Banco_emisor"] = 'BANAMEX'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '1.79', "Banco_emisor"] = 'BANAMEX'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '0', "Banco_emisor"] = 'SANTANDER - REFERENCIA NUMERICA'
rs2.loc[rs2['disbursement_tracking_key'].str[:4] == '#¡REF!', "Banco_emisor"] = 'CEP PENDIENTE DE ENVIO'
rs2.loc[rs2['Banco_emisor'] == 'BANAMEX', "Banco emisor"] = '40002'
rs2.loc[rs2['Banco_emisor'] == 'STP', "Banco emisor"] = '90646'
rs2.loc[rs2['Banco_emisor'] == 'SANTANDER', "Banco emisor"] = '40014'
rs2['Comision'] = ''
rs2['LINEA CEP'] = (rs2['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['disbursement_tracking_key'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['Banco emisor'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['CLABE'].astype(str).replace(' ', '',regex=True)+','
                    +rs2['amount'].astype(str).replace(' ', '',regex=True))
rs2.head()

In [ ]:
rs2.shape

# ARCHIVO DISPERSION

In [ ]:
disp['date_disbursement'] = date_respuesta
disp['report_id'] = disp['report_id'].str.strip()
disp['report_date_local'] = pd.to_datetime(disp['report_date_local'])

In [ ]:
disp.loc[disp['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp.loc[disp['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp.drop(['bank_name','bank_account_number','status','swap','payment_date_local'],axis=1,inplace=True)
disp.head()

In [ ]:
disp.shape

In [ ]:
disp_2 = disp.merge(rs2[['report_id','status','Nombre de la institución','CLABE','LINEA CEP','Banco_emisor']]
                    ,on='report_id', how='left')
disp_2.loc[disp_2['status'] != 'PAID', 'status'] = 'ERROR'
disp_2.sort_values(by=['status'])
disp_2.head()

In [ ]:
disp_2.loc[(disp_2['Nombre de la institución'] == 'SANTANDER') &
           (disp_2['Banco_emisor'] == 'SANTANDER'), 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO SANTANDER'
disp_2.loc[(disp_2['Nombre de la institución'] == 'BANAMEX') &
           (disp_2['Banco_emisor'] == 'BANAMEX'), 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO BANAMEX'
disp_2.loc[(disp_2['Nombre de la institución'] == 'STP') &
           (disp_2['Banco_emisor'] == 'STP'), 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO STP'
# disp_2['LINEA CEP'] = (np.where(disp_2['Banco_emisor'] == 'SANTANDER - REFERENCIA NUMERICA',
#                                 disp_2['Referencia Numerica'], disp_2['LINEA_CEP']))
disp_2.loc[disp_2['status'] != 'PAID', 'LINEA CEP'] = ''
disp_2.loc[disp_2['status'] != 'PAID', 'Banco_emisor'] = ''
disp_2.loc[disp_2['LINEA CEP'] == 'CEP PENDIENTE DE ENVIO', 'Banco_emisor'] = ''

In [ ]:
for col in disp_2.columns: 
    print(col) 

In [ ]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','bank_account_owner_name','CLABE','status','LINEA CEP','Banco_emisor']]
disp_final.rename(columns ={'Nombre de la institución':'bank','amount':'total_due'}, inplace=True)
disp_final.sort_values(by=['status'], ascending=True, inplace=True)
disp_final.head()

In [ ]:
disp_final.shape

In [ ]:
duplicates_disp_final = disp_final[disp_final.duplicated(['report_id'],keep=False)]
duplicates_disp_final.head()

In [ ]:
duplicates_disp_final.shape

# ERROR

In [ ]:
error = disp_final[(disp_final['status'] != 'PAID')]
error.head()

In [ ]:
error.shape

In [ ]:
error.drop(['bank_account_owner_name','CLABE','status','LINEA CEP','Banco_emisor'],axis=1,inplace=True)

In [ ]:
error.loc[error['bank'] == 'SANTANDER', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'SANTANDER', 'Check'] = 'SPEI DEVUELTO'

In [ ]:
error_final = error.merge(rs2[['report_id','error_description']] ,on='report_id', how='left')
error_final.head()

In [ ]:
error_final.shape

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [ ]:
writer = pd.ExcelWriter('DISPERSION SWAP.xlsx', engine='xlsxwriter', engine_kwargs={'options': {'encoding':'utf-8'}})
disp_final.to_excel(writer, sheet_name='SWAP DEL DIA',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.close()

### ARCHIVO RESPUESTA

In [ ]:
rs2.head()

In [ ]:
rs_final = rs2[['disbursement_tracking_key','bank_account_owner_name','report_id','CLABE',
                'Nombre de la institución','amount','Comision','status','error_description','Date_respuesta',
                'Metodo de dispersion']]
rs_final.rename(columns = {'disbursement_tracking_key':'Rastreo CEP'
                           ,'bank_account_owner_name':'Nombre','CLABE':'Cuenta'
                           ,'Nombre de la institución':'Banco','amount':'Importe','status':'Estatus'
                           ,'error_description':'Mensaje'}, inplace=True)
rs_final['Cuenta'] = "'"+rs_final['Cuenta']
rs_final.sort_values(by=['Estatus'], ascending=True, inplace=True)
rs_final.head()

In [ ]:
rs_final.shape

In [ ]:
rs_final.to_csv(r'G:\My Drive\Settlement\Respuestas Swap\2022\02. Febrero 2022\Respuesta 2022-02-09.csv',index=False)